In [ ]:
import os
import json
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt

In [ ]:
IMAGE_SIZE = (100, 100)
BATCH_SIZE = 32
EPOCHS = 10
AUTOTUNE = tf.data.AUTOTUNE
TRAIN_DIR = "../dataset/Training"
TEST_DIR = "../dataset/Test"
MODEL_SAVE_PATH = "../model/food_classifier.h5"
CLASS_INDEX_PATH = "../model/class_indices.json"

In [ ]:
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0  # normalize to [0, 1]
    return image, label

In [ ]:
def load_dataset(data_dir, shuffle=True, cache_name=None):
    dataset = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        image_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        label_mode="categorical",
        shuffle=shuffle,
        seed=42
    )

    class_names = dataset.class_names
    print(f"Loaded {len(class_names)} classes from {data_dir}")

    dataset = (
        dataset
        .map(preprocess, num_parallel_calls=AUTOTUNE)
        .cache(cache_name)
        .shuffle(1000) if shuffle else dataset
    )
    return dataset.prefetch(AUTOTUNE), class_names

In [ ]:
cache_dir = "../cache"
if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)

In [ ]:
print("Loading datasets...")
train_ds, class_names = load_dataset(TRAIN_DIR, cache_name="../cache/train_cache")
val_ds, _ = load_dataset(TEST_DIR, shuffle=False, cache_name="../cache/val_cache")

In [ ]:
os.makedirs("model", exist_ok=True)
class_indices = {name: i for i, name in enumerate(class_names)}
with open(CLASS_INDEX_PATH, "w") as f:
    json.dump(class_indices, f)

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(*IMAGE_SIZE, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(class_names), activation='softmax')
])

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
print("Training model...")
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

In [ ]:
model.save(MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")

In [ ]:
def plot_training(history):
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"], label="Train Acc")
    plt.plot(history.history["val_accuracy"], label="Val Acc")
    plt.title("Accuracy")
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Val Loss")
    plt.title("Loss")
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig("../model/training_plot.png")
    plt.show()

plot_training(history)
